# [180. Consecutive Numbers](https://leetcode.com/problems/consecutive-numbers/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Logs

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| num         | varchar |
+-------------+---------+</pre>
In SQL, id is the primary key for this table.
id is an autoincrement column.
 

Find all numbers that appear at least three times consecutively.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Logs table:
<pre>+----+-----+
| id | num |
+----+-----+
| 1  | 1   |
| 2  | 1   |
| 3  | 1   |
| 4  | 2   |
| 5  | 1   |
| 6  | 2   |
| 7  | 2   |
+----+-----+</pre>
Output: 
<pre>+-----------------+
| ConsecutiveNums |
+-----------------+
| 1               |
+-----------------+</pre>
Explanation: 1 is the only number that appears consecutively for at least three times.

In [1]:
#pandas schema

import pandas as pd

data = [[1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 2]]
logs = pd.DataFrame(data, columns=['id', 'num']).astype({'id':'Int64', 'num':'Int64'})

In [2]:
# to spark schema

import pyspark.sql.functions as F
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

logs_df = spark.createDataFrame(logs)
logs_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/05 12:27:12 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/05 12:27:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 12:27:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+---+
|id |num|
+---+---+
|1  |1  |
|2  |1  |
|3  |1  |
|4  |2  |
|5  |1  |
|6  |2  |
|7  |2  |
+---+---+


In [3]:
# in Spark Dataframe
from pyspark.sql.window import Window

logs_df.\
    withColumn('leadd',F.lead('num').over(Window.orderBy('id'))).\
    withColumn('lagg',F.lag('num').over(Window.orderBy('id'))).\
    filter((F.col('leadd')==F.col('lagg')) & (F.col('lagg')==F.col('num'))).\
    select(F.col('num').alias('ConsecutiveNums')).show()
    

23/11/05 12:27:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/05 12:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/05 12:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/05 12:27:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/05 12:27:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------------+
|ConsecutiveNums|
+---------------+
|              1|
+---------------+


In [4]:
# in Spark SQL
logs_df.createOrReplaceTempView('logs')
spark.sql('''
select distinct num as ConsecutiveNums from 
(select id,num,lead(num) over (order by id) as leadd,lag(num) over (order by id) as lagg from logs) l
where num=leadd and num=lagg;
''').show()

23/11/05 12:27:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/05 12:27:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/05 12:27:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/05 12:27:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------------+
|ConsecutiveNums|
+---------------+
|              1|
+---------------+


In [5]:
spark.stop()